In [1]:
import librosa
import librosa.display

import cv2
import os

import numpy as np
import matplotlib.pyplot as plt
cv2.__version__

'4.5.5'

In [2]:
y = np.load('test.npz')['sig']
sr = 44100
y.shape

(441000,)

In [3]:
hop_len = 512
freq_w = 3
octave = 8

C = librosa.cqt(y, sr=sr, hop_length= hop_len, n_bins= 12*freq_w*octave, bins_per_octave= 12*freq_w)
C_db = librosa.amplitude_to_db(np.abs(C),ref=np.max)+80
c_compression = (C_db/80*255).astype(np.uint8)
print(C_db.shape)
print(c_compression.min(),c_compression.max())

(288, 862)
0 255


In [4]:
gaussian_mask = np.array([[1/16,1/8,1/16], [1/8,1/4,1/8], [1/16,1/8,1/16]])
sharpening_mask1 = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
sharpening_mask2 = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])

In [ ]:
def nothing(pos):
    pass

src = c_compression

cv2.namedWindow('spectrum')
cv2.createTrackbar('min','spectrum',0,255,nothing)

cv2.imshow('spectrum', src)
img = src.copy()
tmp = []
while True:
    tmp.append(img.copy())
    
    kw=cv2.waitKey()
    if kw==27:
        break
    elif kw==ord('z'):
        img = tmp.pop()
        print('실행취소')
    elif kw==ord('r'):
        img = src.copy()
        print()
        print('-'*10)
        print("Reset")
        print('-'*10)
        print()
    elif kw==ord('t'):
        pos = cv2.getTrackbarPos('min','spectrum')
        img = np.where(img<pos,0,img)
        print("%d under 0"%pos)
    elif kw==ord('s'):
        i_min = img.min()
        i_max = img.max()
        img = 255/(i_max-i_min)*(img-i_min)
        print("stretching")
    elif kw==ord('g'):
        img = cv2.filter2D(img,-1,gaussian_mask)
        print("Gaussian filter")
    elif kw==ord('1'):
        img = cv2.filter2D(img,-1,sharpening_mask1)
        print("8 connected sharpening")
    elif kw==ord('2'):
        img = cv2.filter2D(img,-1,sharpening_mask2)
        print("4 connected sharpening")
        
    cv2.imshow('spectrum', img)
        

cv2.destroyAllWindows()